In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.stats as ss

import sklearn.model_selection as ms 

In [2]:
# load your data (don't touch, just run)

"""
data = []
f = open('covtype.data','r')
while(1):
    line = f.readline()
    if  len(line) < 100:
        print(line)
    
    if len(line) == 0: break
    data.append(np.array([float(k) for k in line.split(',')]))
    if len(data) % 100000 == 0:
        print(len(data))
        
f.close
data = np.vstack(data)
N = data.shape[0]
idx = np.random.permutation(N)


X_test = data[:N//5,:]
X_train = data[N//5:,:]
y_test = X_test[:,-1]
y_train = X_train[:,-1]
X_test = X_test[:,:-1]
X_train = X_train[:,:-1]


sio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})

data = sio.loadmat('covtype.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

y_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]

for i in range(len(y_idx_train)):
    y_idx = y_idx_train[i]
    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)//1000+1,replace=False)]
    
y_idx_train = np.hstack(y_idx_train)
y_idx_train = np.random.permutation(y_idx_train)

X_train = X_train[y_idx_train,:]
y_train = y_train[y_idx_train]

sio.savemat('covtype_reduced.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})
"""




100000
200000
300000
400000
500000



In [3]:

data = sio.loadmat('covtype_reduced.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

print(np.unique(y_train), np.unique(y_test))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


[1. 2. 3. 4. 5. 6. 7.] [1. 2. 3. 4. 5. 6. 7.]
(468, 54) (116202, 54) (468,) (116202,)


In [4]:
def entropy(label):
    entropy = 0
    for i in np.unique(label):
        p = np.sum(label == i)/float(len(label))
        entropy += -p*np.log2(p)
    return entropy


def cond_entropy(label1, split):
    cond_entropy = 0
    for i in np.unique(label1):
        for j in np.unique(split):
            probIandJ = np.sum(np.logical_and(label1 == i, split == j))/float(len(label1))
            probIGivenJ = np.sum(np.logical_and(label1 == i, split == j))/float(np.sum(split == j))
            if probIGivenJ != 0:
                cond_entropy += -probIandJ*np.log2(probIGivenJ)
    return cond_entropy


random_sequences = sio.loadmat('random_sequences.mat')

s1 = random_sequences['s1'][0]
s2 = random_sequences['s2'][0]
# print(f'S1: {s1}')
# print(f'S2: {s2}')

print('entropy = ', entropy(s1))
print('conditional entropy = ', cond_entropy(s1, s2))


entropy =  3.314182323161083
conditional entropy =  3.3029598816135173


In [5]:
def find_best_split(x, y):
    best_feat = 0
    splitval = 0.
    set1 = range(int(len(y) / 2))
    set2 = range(int(len(y) / 2), len(y))

    set1ToReturn = set1
    set2ToReturn = set2
    # find the best feature to split on and the best split value
    # based on information gain

    max_info_gain = -float("inf")
    XTranspose = x.T
    for i in range(len(XTranspose)):
        for j in np.unique(XTranspose[i]):
            set1 = np.where(XTranspose[i] < j)[0]
            set2 = np.where(XTranspose[i] >= j)[0]
            SlOverS = len(set1) / float(len(y))
            SrOverS = len(set2) / float(len(y))
            info_gain = entropy(y) - (SlOverS * entropy(y[set1]) + SrOverS * entropy(y[set2]))
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feat = i
                splitval = j
                set1ToReturn = set1
                set2ToReturn = set2

    # print(f'Max info gain: {max_info_gain}')
    # print(f'Best feature: {best_feat}')
    # print(f'Split value: {splitval}')
    # print(f'Set 1: {set1ToReturn}')
    # print(f'Set 2: {set2ToReturn}')
    return best_feat, splitval, set1ToReturn, set2ToReturn


best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
y_new = y_train * 0
y_new[set1] = 1
print('information gained in first step', entropy(
    y_train) - cond_entropy(y_train, y_new))
print(best_feat, splitval)

information gained in first step 0.35686227423621664
0 2842.0


In [6]:
def purity(y):
    return ss.mode(y, keepdims=True)[1] / len(y + 0.)


class Node:
    def __init__(self, sample_idx, nodeid, is_leaf=True):
        self.is_leaf = is_leaf
        self.id = nodeid
        self.sample_idx = sample_idx
        self.children = []
        self.split_feat = None
        self.split_val = None

    def visit_node(self, x):
        if self.is_leaf:
            return self.label
        else:
            if x[self.split_feat] < self.split_val:
                return self.children[0].visit_node(x)
            else:
                return self.children[1].visit_node(x)

    def add_split_details(self, splitfeat, splitval):
        self.split_feat = splitfeat
        self.split_val = splitval


class Tree:
    def __init__(self, x, y):
        m = len(y)
        self.x = x
        self.y = y
        self.maxid = -1
        self.root = self.construct_node(np.array(range(m)))
        self.leaves = [self.root]

    def print_tree(self):
        print('printing tree...')

        def print_node(parent, node):
            print(node.id, end='')

            if parent is not None:
                print(f', parent: {parent.id}', end='')
            else:
                print(f', ROOT', end='')

            print(', label ', node.label, end='')
            if node.is_leaf:
                print(', LEAF, ', 'nsamples %d, purity %.2f' % (len(node.sample_idx), purity(self.y[node.sample_idx])))
            else:
                print(', NONLEAF, split %d, val %.2f' % (node.split_feat, node.split_val))
            if not node.is_leaf:
                for ch in node.children:
                    print_node(node, ch)
        print_node(None, self.root)

    def construct_node(self, sample_idx):
        node = Node(sample_idx, self.maxid + 1, True)
        node.label = ss.mode(self.y[sample_idx], keepdims=True)[0][0]
        node.entropy = entropy(self.y[sample_idx])
        node.num_mistakes = np.sum(np.not_equal(node.label, self.y[sample_idx]))
        self.maxid += 1
        return node

    def report_train_err(self):
        total_mistakes = 0
        for leaf in self.leaves:
            total_mistakes += leaf.num_mistakes
        return total_mistakes / (len(self.y) + 0.)

    def predict(self, x):
        return self.root.visit_node(x)


In [9]:
def get_test_err(tree):
    # get test error
    num_test_mistakes = 0
    for k in range(len(y_test)):
        x,y = X_test[k,:],y_test[k]
        if y != tree.predict(x):
            num_test_mistakes += 1
    return num_test_mistakes / (len(y_test)+0.)



tree = Tree(X_train,y_train)
tree.print_tree()
print('current train err:', tree.report_train_err())
print('current test err:', get_test_err(tree))


# my first split
best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
 
left_child = tree.construct_node(set1)
right_child = tree.construct_node(set2)
tree.root.is_leaf = False
tree.leaves.pop(tree.leaves.index(tree.root))
tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)


tree.root.children = [left_child, right_child]
tree.leaves.extend(tree.root.children)
tree.print_tree()
print('one step train err:', tree.report_train_err())
print('one step test err:', get_test_err(tree))


# Run 24 more iterations for a total of 25 iterations of the decision tree algorithm
i = 0
while(i < 24):
    leaf = tree.leaves.pop(0)
    # If the purity is 1, then the node is a leaf node and you skip it
    if purity(y_train[leaf.sample_idx]) == 1:
        continue
    # Now we split
    best_feat, splitval, set1, set2 = find_best_split(X_train[leaf.sample_idx, :], y_train[leaf.sample_idx])
    # Split the leaf node into two child nodes
    left_child = tree.construct_node(leaf.sample_idx[set1])
    right_child = tree.construct_node(leaf.sample_idx[set2])
    leaf.is_leaf = False
    leaf.add_split_details(splitfeat=best_feat, splitval=splitval)
    leaf.children = [left_child, right_child]
    tree.leaves.extend(leaf.children)
    i = i + 1

print('final train err:', tree.report_train_err())
print('final test err:', get_test_err(tree))
tree.print_tree()


printing tree...
0, ROOT, label  2.0, LEAF,  nsamples 468, purity 0.44
current train err: 0.5641025641025641
current test err: 0.3138069912738163
printing tree...
0, ROOT, label  2.0, NONLEAF, split 0, val 2657.00
1, parent: 0, label  3.0, LEAF,  nsamples 73, purity 0.47
2, parent: 0, label  1.0, LEAF,  nsamples 395, purity 0.47
one step train err: 0.5299145299145299
one step test err: 0.767998829624275
final train err: 0.24786324786324787
final test err: 0.30195693705788196
printing tree...
0, ROOT, label  2.0, NONLEAF, split 0, val 2657.00
1, parent: 0, label  3.0, NONLEAF, split 0, val 2445.00
3, parent: 1, label  3.0, NONLEAF, split 6, val 162.00
7, parent: 3, label  3.0, NONLEAF, split 1, val 350.00
15, parent: 7, label  3.0, LEAF,  nsamples 10, purity 1.00
16, parent: 7, label  2.0, LEAF,  nsamples 1, purity 1.00
8, parent: 3, label  3.0, NONLEAF, split 3, val 342.00
17, parent: 8, label  6.0, NONLEAF, split 6, val 242.00
31, parent: 17, label  6.0, LEAF,  nsamples 13, purity 0.7

In [10]:
"""
# Run a few more iterations of the decision tree until the training error is 0
i = 25
while(tree.leaves):
    leaf = tree.leaves.pop(0)
    # If the purity is 1, then the node is a leaf node and you skip it
    if purity(y_train[leaf.sample_idx]) == 1:
        continue
    # Now we split
    best_feat, splitval, set1, set2 = find_best_split(X_train[leaf.sample_idx, :], y_train[leaf.sample_idx])
    # Split the leaf node into two child nodes
    left_child = tree.construct_node(leaf.sample_idx[set1])
    right_child = tree.construct_node(leaf.sample_idx[set2])
    leaf.is_leaf = False
    leaf.add_split_details(splitfeat=best_feat, splitval=splitval)
    leaf.children = [left_child, right_child]
    tree.leaves.extend(leaf.children)
    i = i + 1

print(f'Took {i} iterations in total to get to 0 training error with {i-25} coming from this loop')
print('final train err:', tree.report_train_err())
print('final test err:', get_test_err(tree))
tree.print_tree()
"""

Took 103 iterations in total to get to 0 training error with 78 coming from this loop
final train err: 0.0
final test err: 0.48542193766028124
printing tree...
0, ROOT, label  2.0, NONLEAF, split 0, val 2657.00
1, parent: 0, label  3.0, NONLEAF, split 0, val 2445.00
3, parent: 1, label  3.0, NONLEAF, split 6, val 162.00
7, parent: 3, label  3.0, NONLEAF, split 1, val 350.00
15, parent: 7, label  3.0, LEAF,  nsamples 10, purity 1.00
16, parent: 7, label  2.0, LEAF,  nsamples 1, purity 1.00
8, parent: 3, label  3.0, NONLEAF, split 3, val 342.00
17, parent: 8, label  6.0, NONLEAF, split 6, val 242.00
31, parent: 17, label  6.0, NONLEAF, split 2, val 12.00
57, parent: 31, label  2.0, NONLEAF, split 0, val 2362.00
81, parent: 57, label  3.0, LEAF,  nsamples 1, purity 1.00
82, parent: 57, label  2.0, LEAF,  nsamples 1, purity 1.00
58, parent: 31, label  6.0, NONLEAF, split 0, val 2242.00
83, parent: 58, label  3.0, LEAF,  nsamples 1, purity 1.00
84, parent: 58, label  6.0, LEAF,  nsamples 10